# HPMC/TFA Oscillations

In [75]:
import numpy as np
from scipy.signal import convolve2d
import matplotlib.pyplot as plt
from matplotlib import animation
from ipywidgets import *

$2X \rightarrow Y$

$Y + B \rightarrow Z$

$Z \rightarrow D + E + X$

In [76]:
def update(t,arr,er):
    """Update arr[p] to arr[q] by evolving in time."""

    # Count the average amount of each species in the 9 cells around each cell
    # by convolution with the 3x3 array m.
    s = np.zeros((ns, ny, nx))
    m = np.ones((3,3)) * dRate # Diffusing species
    m[1,1] = 1-8*dRate
    m2 = np.zeros((3,3))    # Non-diffusing species
    m2[1,1] = 1
    
    for k in range(ns):
        if k in (1,2,3):
            s[k] = convolve2d(arr[t,k], m, mode='same', boundary='wrap')
        elif k == 0:
            s[k] = convolve2d(arr[t,k], m, mode='same', boundary='fill',
             fillvalue = er*arr[0,k]) # NEED TO CHECK THIS'''
        else:
            s[k] = convolve2d(arr[t,k], m2, mode='same', boundary='wrap')
    
    # Apply the reaction equations

    arr[t+1,0] = s[0]
    arr[t+1,1] = s[1] + alpha*(-pow(s[1],2)+s[3])
    arr[t+1,2] = s[2] + alpha*(pow(s[1],2)-s[2]*s[0])
    arr[t+1,3] = s[3] + alpha*(s[2]*s[0]-s[3])
     
    '''if t == 200:
        arr[q,0] = 0.8
        arr[q,1] = 0.8'''
    
    # Ensure the species concentrations are kept within [0,1]. (<0)=0; (>1)=1
    #np.clip(arr[q,3], 0, 0.5, arr[q,3])

    return arr

In [77]:
ITER = 200
nx, ny = 20, 20              # Width, height of the image.
alpha = 0.1                    # Reaction parameter.
ns = 4                         # Number of chemical species

''' Diffusion '''
dRate = 1/9 # dRate in neighbours (dCenter+8*dRate = 1) dRate <= 1/8

''' Evaporation '''
er = 1.0 # Evaporation rate 0 >= er >= 1

def slider1(B, X, Y, Z):

    ic = [B, X, Y, Z]
    # Initialize the array with default values
    #arr = np.random.random(size=(2, ns, ny, nx)) # random values
    arr = np.zeros((ITER,ns,ny,nx))
    for i in range(ns): arr[0, i] = ic[i]

    # PLOT MID
    label = ['$\mathrm{B}$','$\mathrm{X}$',
             '$\mathrm{Y}$','$\mathrm{Z}$']

    fig, ax = plt.subplots(1, 4)
    fig.subplots_adjust(hspace = 0.4)
    c = 0
    for i in ax.flat:
        i.set(xlabel=label[c])
        c += 1

    for i in range(ITER-1):
        arr = update(i, arr, er)

    time = np.linspace(1,ITER,ITER)
    for j in range(ns):
            ax[j].plot(time, arr[:, j, int(ny/2),int(nx/2)], color='black')
            ax[j].plot(time, arr[:, j, 0,int(nx/2)], color='blue')

interact(slider1, B=widgets.FloatSlider(min=0,max=1,step=0.1,value=0.1), X=widgets.FloatSlider(min=0,max=1,step=0.1,value=0.8), 
         Y=widgets.FloatSlider(min=0,max=1,step=0.1,value=0.5), Z=widgets.FloatSlider(min=0,max=1,step=0.1,value=0.4))

interactive(children=(FloatSlider(value=0.1, description='B', max=1.0), FloatSlider(value=0.8, description='X'…

<function __main__.slider1(B, X, Y, Z)>

In [78]:
# Set up the image ############################################################

def slider(i = 1):
    fig, ax = plt.subplots()
    im = ax.imshow(arr[i].reshape((ns*nx,ny)).T,
               cmap=plt.cm.jet, vmin = 0, vmax = 1)
    ax.set_xticks(nx*(np.linspace(1,ns,ns)-.5))
    # MANUAL EDIT #################################################################
    ax.set_xticklabels(['$\mathrm{B}$','$\mathrm{X}$',
                        '$\mathrm{Y}$','$\mathrm{Z}$',
                        '$\mathrm{X}$','$\mathrm{Cell_{TFA}}$',
                        '$\mathrm{H_2O}$'])
    ###############################################################################
    ax.axes.get_yaxis().set_visible(False)

    aspect = 5
    pad_fraction = 0.5
    divider = make_axes_locatable(ax)
    width = axes_size.AxesY(ax, aspect=1./aspect)
    pad = axes_size.Fraction(pad_fraction, width)
    cax = divider.append_axes("right", size=width, pad=pad)
    plt.colorbar(im,cax=cax)
    # Update the image for iteration i of the Matplotlib animation.
    #im.set_array(arr[i].reshape((ns*nx,ny)).T)
    #fig.canvas.draw_idle()
    
interact(slider, i=widgets.IntSlider(min=0,max=ITER-1,step=2,));

# To save the animation as an MP4 movie, uncomment this line ##################

#anim.save(filename='bz_temp.mp4', fps=25)

interactive(children=(IntSlider(value=0, description='i', max=199, step=2), Output()), _dom_classes=('widget-i…